# Info bici

In [30]:
import json
from shapely import MultiLineString, LineString, Point

import geopandas as gpd

from my_paths import *

## Piste ciclabili

Carichiamo il file Raw geojson

In [4]:
gdf_ciclabili_raw = gpd.read_file(PATH_CICLABILI_RAW)

In [51]:
def group_by_id_via_geometry(geos):
    new_geo = []
    for geo in geos:
        new_geo.append(geo)
    return MultiLineString(new_geo)

In [58]:
gdf_ciclabili_staging = gdf_ciclabili_raw.copy()

# Convertiamo alcuni tipi
gdf_ciclabili_staging["geometry"] = gdf_ciclabili_staging["geometry"].apply(lambda x: x.geoms[0])
gdf_ciclabili_staging["fine_lavor"] = gdf_ciclabili_staging["fine_lavor"].astype(int)
gdf_ciclabili_staging["lunghezza"] = gdf_ciclabili_staging["lunghezza"].astype(int)


gdf_ciclabili_staging = gdf_ciclabili_staging.drop(["id_amat", "gerarchia"], axis=1)
gdf_ciclabili_staging = gdf_ciclabili_staging.groupby("id_via").agg({
    "tipologia": lambda x: str(set(x)),
    "fine_lavor": "max",
    "lunghezza": "sum",
    "geometry": group_by_id_via_geometry
})
gdf_ciclabili_staging = gpd.GeoDataFrame(gdf_ciclabili_staging, geometry="geometry")

Salviamo il file modificato geojson in Clean

In [59]:
gdf_ciclabili_clean = gdf_ciclabili_staging.copy()
gdf_ciclabili_clean.to_file(PATH_CICLABILI_CLEAN, driver="GeoJSON")

c:\Matteo\Coding\Project\Progetto Esame BI\.venv\Lib\site-packages\pyogrio\geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


## Stazioni BikeMi

In [ ]:
gdf = gpd.read_file(PATH_BIKEMI_RAW, sep = ';')
gdf = gdf.rename(columns = {'id_amat':'id_stazione',
                           'stalli':'n_posti',
                           'LONG_X_4326':'longitudine',
                           'LAT_Y_4326': 'latitudine',
                           'Location':'location'})
gdf.drop(['stato', 'id_via', 'indirizzo', 'civico', 'zd_attuale', 'anno', "tipo"], axis=1, inplace=True)
gdf['icon'] = "share"
gdf["geometry"] = gdf.apply(lambda row: Point(row['longitudine'], row['latitudine']), axis=1)
gdf = gdf.set_geometry("geometry", crs=4326)
gdf = gdf.drop(columns="location")

In [15]:
gdf.to_file(PATH_BIKEMI_CLEAN, driver="GeoJSON")